##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FOOD1 with TensorFlow Hub - TFLite

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%202%20-%20TensorFlow%20Lite/Week%202/Exercise/TFLite_Week2_Exercise_Answer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%202%20-%20TensorFlow%20Lite/Week%202/Exercise/TFLite_Week2_Exercise_Answer.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

## Setup

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range
import sklearn.metrics


import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm

print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

## Select the Hub/TF2 Module to Use

Hub modules for TF 1.x won't work here, please use one of the selections provided.

In [ ]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

## Data Preprocessing

Use [TensorFlow Datasets](http://tensorflow.org/datasets) to load the cats and dogs dataset.

This `tfds` package is the easiest way to load pre-defined data. If you have your own data, and are interested in importing using it with TensorFlow see [loading image data](../load_data/images.ipynb)


In [ ]:
import tensorflow_datasets as tfds
#tfds.disable_progress_bar()

The `tfds.load` method downloads and caches the data, and returns a `tf.data.Dataset` object. These objects provide powerful, efficient methods for manipulating data and piping it into your model.

Since `"cats_vs_dog"` doesn't define standard splits, use the subsplit feature to divide it into (train, validation, test) with 80%, 10%, 10% of the data respectively.

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#splits = tfds.Split.(weighted=(80, 10, 10))

splits, info = tfds.load('food101', with_info=True, as_supervised=True, split = ['train' ,'validation[:50%]' ,'validation[50%:]'])







In [ ]:
(train_examples, validation_examples, test_examples) = splits

In [ ]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes




print(num_classes)

In [ ]:
class_names = np.array(info.features['label'].names)

In [ ]:
print(class_names)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline



In [ ]:
get_label_name = info.features['label'].int2str
#for i in range(num_classes):
 #print(get_label_name(i))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for image, label in train_examples.take(4):
  #print(image.shape)
  gray = cv2.cvtColor(image.numpy(), cv2.COLOR_RGB2GRAY)
  # Try Canny using "wide" and "tight" thresholds
  wide = cv2.Canny(gray, 30, 100)
  tight = cv2.Canny(gray, 200, 240)
 
  ax1.set_title('wide')
  ax1.imshow(wide, cmap='gray')

  ax2.set_title('tight')
  ax2.imshow(tight, cmap='gray')
    
# Display the images


  plt.figure()
  plt.imshow(image)
  print(label)
  plt.title(get_label_name(label))

In [ ]:
train_label = [ label for image, label in train_examples]

In [ ]:

#print(train_images)

# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(20,20))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_label[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow([image for image, label in train_examples.take(28)][i] )
  
  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)

#

In [ ]:
%tensorboard --logdir logs/plots

### Format the Data

Use the `tf.image` module to format the images for the task.

Resize the images to a fixes input size, and rescale the input channels

In [ ]:
def format_image(image, label):
    image = tf.cast(image, tf.float32)
    #image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, IMAGE_SIZE) #/ 255.0
    #print(image.get_shape)
    return  image, label

Now shuffle and batch the data


In [ ]:
BATCH_SIZE =  128#@param {type:"integer"}

In [ ]:
train_batches = train_examples.shuffle(1000).map(format_image).batch(BATCH_SIZE).prefetch(1)


validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

cm_validation_batches = validation_examples.map(format_image).batch(2160).prefetch(1)


test_batches = test_examples.batch(1).map(format_image)

In [ ]:
print(0.90*2400)

Inspect a batch

In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

In [ ]:

a,  = cm_validation_batches.take(1)
im , l = a[0], a[1]




print(im.shape)

In [ ]:
from matplotlib.colors import Normalize
import matplotlib.cm  as cm


def class_distribution(train_examples , validation_examples , test_examples):
        train_label_plot = [ label for image, label in train_examples]
        valid_label_plot = [ label for image, label in validation_examples]
        test_label_plot = [ label for image, label in test_examples]
        unique, counts = np.unique(train_label_plot, return_counts=True)
        #print(unique)

        my_cmap = cm.get_cmap('jet')
        plt.figure(figsize=(20,70))
        # Get normalize function (takes data in range [vmin, vmax] -> [0, 1])
        my_norm = Normalize(vmin=0, vmax=196)
        plt.barh(unique, counts ,color=my_cmap(my_norm(unique)))

        plt.yticks(unique, class_names)
        plt.title('Class Frequency')
        plt.xlabel('Class')
        plt.ylabel('Frequency')
        plt.show()

        fig = plt.gcf

        return fig

class_distribution(train_examples , validation_examples , test_examples)

## Defining the Model

All it takes is to put a linear classifier on top of the `feature_extractor_layer` with the Hub module.

For speed, we start out with a non-trainable `feature_extractor_layer`, but you can also enable fine-tuning for greater accuracy.

In [ ]:
do_fine_tuning = True #@param {type:"boolean"}

In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=do_fine_tuning)


In [ ]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        #tf.keras.layers.Dense(1280, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
#@title (Optional) Unfreeze some layers
NUM_LAYERS = 3 #@param {type:"slider", min:1, max:50, step:1}
      
if do_fine_tuning:
    feature_extractor.trainable = False
    
    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

In [ ]:
model.summary()

## Training the Model

In [ ]:
if do_fine_tuning:
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
else:
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7 , mode = 'min')
model.summary()

In [ ]:
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')
file_writer_roc = tf.summary.create_file_writer(logdir + '/roc')

In [ ]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [ ]:

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(im)
  test_pred = np.argmax(test_pred_raw, axis=1)

      # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(l, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot as skplt

def plot_roc(y_true, y_probas):
  #figure = plt.figure(figsize=(8, 8))
  figure, axes = plt.subplots(1,1, figsize = (8,8))

  skplt.metrics.plot_roc_curve(y_true, y_probas , ax =axes ,text_fontsize ='small', figsize= (8 ,8))
  plt.title('ROC')
  #plt.colorbar()
  plt.ylabel('True Label')
  plt.xlabel('Predicated Label')
  fig = plt.gcf()
  #plt.show()
  fig.savefig("test_rasterization.png", dpi=150)
  #print(fig)
  

  return fig

In [ ]:
def log_roc(epoch, logs):
  test_pred_raw = model.predict(im)
  test_pred = np.argmax(test_pred_raw, axis=1)
  #print(test_pred.shape)
  #print(l.shape)
  figure_roc = plot_roc(l, test_pred_raw)
  roc_image = plot_to_image(figure_roc)


  # Log the roc  as an image summary.
  with file_writer_roc.as_default():
    tf.summary.image("ROC", roc_image, step=epoch)

roc_callback = tf.keras.callbacks.LambdaCallback( on_epoch_end=log_roc)

In [ ]:
cc



```
# learning rate dense nodes accuacy 
#  0.0002           101      16%
#  0.002            1280     15%
```



In [ ]:
EPOCHS = 40
# Start TensorBoard.


#%tensorboard --logdir logs/



hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 callbacks =[early_stop_callback, reduce_lr],
                 validation_data=validation_batches,
                 validation_steps = 10)
                 #callbacks=[tensorboard_callback])

In [ ]:
from matplotlib import pyplot as plt




fig, axs = plt.subplots(2, 1 , figsize=(10,5))
#axs[0, 0].plot(x, y)
##axs[0, 0].set_title('Axis [0, 0]')
#axs[0, 1].plot(x, y, 'tab:orange')
#axs[0, 1].set_title('Axis [0, 1]')
#axs[1, 0].plot(x, -y, 'tab:green')
#axs[1, 0].set_title('Axis [1, 0]')
#axs[1, 1].plot(x, -y, 'tab:red')
#axs[1, 1].set_title('Axis [1, 1]')



#plt.yscale('log')
axs[0].plot(hist.history['loss'])
axs[0].plot(hist.history['val_loss'])
axs[1].plot(hist.history['accuracy'])
axs[1].plot(hist.history['val_accuracy'])


In [ ]:
model.evaluate(test_batches.take(150000))

In [ ]:
model.evaluate(validation_batches.take(10000))

In [ ]:
predictions = []
real_label  = []
for image_batch , labels_batch in validation_batches.take(99):
  
  predictions.extend(np.argmax(model.predict(image_batch), axis =-1))
  real_label.extend(labels_batch.numpy())

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report
plt.figure(figsize = (7,30))
#print("the classification report : \n" , cl_report)
cl_report = classification_report(real_label,predictions, target_names = class_names, output_dict=True)
sns.heatmap(pd.DataFrame(cl_report).iloc[:-1, :].T, annot= True, linewidths=.5)
#print(list(val_ds.class_names))

In [ ]:
import sklearn as sklearn
from sklearn.metrics import confusion_matrix



cm = confusion_matrix(real_label, predictions)
print(cm)

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(40, 40))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm = cm,
                          target_names= class_names,
                          title='Confusion matrix',
                          cmap='inferno_r',
                          normalize=False)

In [ ]:
CLS = class_names

In [ ]:


def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img.numpy().astype("uint8"))

  predicted_label = np.argmax(predictions_array)
  #print(predicted_label)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(CLS[predicted_label],
                                100*np.max(predictions_array),
                                CLS[true_label]),
                                color=color)







def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(101) , rotation =90)
  plt.yticks([])
  thisplot = plt.bar(range(101), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  #print(predicted_label)
  #print(true_label)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:


for imgs , lbs in test_batches.take(2):
  
  pred = model.predict(imgs)
  

In [ ]:

i = 0
plt.figure(figsize=(60,3))
plt.subplot(1,2,1)



plot_image(i, pred[i], lbs, imgs)
plt.subplot(1,2,2)
plot_value_array(i, pred[i],  lbs)
plt.show()

In [ ]:
print(pred.shape)

In [ ]:
cm_test_batches = test_examples.batch(240).map(format_image)

b,  = cm_test_batches.take(1)
test_im , test_l = b[0], b[1]

print(test_im.shape)

In [ ]:
test_pred_raw = model.predict(test_im)
test_pred = np.argmax(test_pred_raw, axis=1)
print("true label:" , len(test_l.numpy()))
print("predicted label:" , len(test_pred))
cl_report = sklearn.metrics.classification_report(test_l.numpy(), test_pred, target_names = class_names)

In [ ]:
print("the classification report : \n" , cl_report)

In [ ]:
skplt.metrics.plot_roc_curve(test_l, test_pred_raw ,text_fontsize ='small', figsize= (8 ,8))

## Export the Model

In [ ]:
RPS_SAVED_MODEL = "rps_saved_model"

Export the SavedModel

In [ ]:
tf.saved_model.save(model, RPS_SAVED_MODEL)

In [ ]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

## Convert Using TFLite's Converter

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(RPS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## Test the TFLite Model Using the Python Interpreter

In [ ]:
# Load TFLite model and allocate tensors.
with open(tflite_model_file, 'rb') as fid:
    tflite_model = fid.read()
    
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
# Gather results for the randomly sampled test images
predictions = []

test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(100)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)

In [ ]:
#@title Utility functions for plotting
# Utilities for plotting

#class_names = ['dandelion',
#'daisy',
#'tulips',
#'sunflowers',
#'roses']

def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    img = np.squeeze(img)
    
    plt.imshow(img[: , : , 0], cmap=plt.cm.binary)
    
    predicted_label = np.argmax(predictions_array)
    
    print(type(predicted_label), type(true_label))
    
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
        
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(predictions_array),
                                         class_names[true_label]), color=color)

In [ ]:
#@title Visualize the outputs { run: "auto" }
index = 81 #@param {type:"slider", min:0, max:100, step:1}
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plot_image(index, predictions, test_labels, test_imgs)
plt.show()

Create a file to save the labels.

In [ ]:
with open('labels.txt', 'w') as f:
    f.write('\n'.join(class_names))

If you are running this notebook in a Colab, you can run the cell below to download the model and labels to your local disk.

**Note**: If the files do not download when you run the cell, try running the cell a second time. Your browser might prompt you to allow multiple files to be downloaded. 

In [ ]:
try:
    from google.colab import files
    files.download('converted_model.tflite')
    files.download('labels.txt')
except:
    pass

# Prepare the Test Images for Download (Optional)

This part involves downloading additional test images for the Mobile Apps only in case you need to try out more samples

In [ ]:
!mkdir -p test_images

In [ ]:
from PIL import Image

for index, (image, label) in enumerate(test_batches.take(50)):
    image = tf.cast(image * 255.0, tf.uint8)
    image = tf.squeeze(image).numpy()
    pil_image = Image.fromarray(image)
    pil_image.save('test_images/{}_{}.jpg'.format(class_names[label[0]], index))

In [ ]:
!ls test_images

In [ ]:
!zip -qq rps_test_images.zip -r test_images/

If you are running this notebook in a Colab, you can run the cell below to download the Zip file with the images to your local disk. 

**Note**: If the Zip file does not download when you run the cell, try running the cell a second time.

In [ ]:
try:
    files.download('rps_test_images.zip')
except:
    pass